In [1]:
import pandas as pd  
import numpy as np  
from prophet import Prophet  
from statsmodels.tsa.arima.model import ARIMA  
from sklearn.preprocessing import MinMaxScaler  
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import LSTM, Dense, Dropout  
import matplotlib.pyplot as plt  
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error  
import joblib  

# CSV dosyasını oku  
file_path = '../Etrn/ai_task_data.csv'  
data = pd.read_csv(file_path)  

# Tarih sütununu datetime formatına çevir  
data['date'] = pd.to_datetime(data['date'])  

# Toplam conversion_count'ı gün bazında grupla  
grouped_data = data.groupby('date').agg({'conversion_count': 'sum'}).reset_index()  

# Özellik mühendisliği  
grouped_data['day_of_week'] = grouped_data['date'].dt.dayofweek  
grouped_data['month'] = grouped_data['date'].dt.month  

# Lag özellikleri  
for lag in [3, 7, 15]:  
    grouped_data[f'lag_{lag}'] = grouped_data['conversion_count'].shift(lag)  

# Hareketli ortalama  
grouped_data['rolling_mean_7'] = grouped_data['conversion_count'].rolling(window=7).mean()  
grouped_data['rolling_mean_15'] = grouped_data['conversion_count'].rolling(window=15).mean()  

# Geçersiz verileri çıkar  
grouped_data.dropna(inplace=True)  

# ----- Prophet Modeli -----  
try:  
    prophet_model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)  
    prophet_model.add_regressor('day_of_week')  
    prophet_model.add_regressor('month')  
    for lag in [3, 7, 15]:  
        prophet_model.add_regressor(f'lag_{lag}')  
    prophet_model.add_regressor('rolling_mean_7')  
    prophet_model.add_regressor('rolling_mean_15')  
    prophet_model.fit(grouped_data.rename(columns={'date': 'ds', 'conversion_count': 'y'}))  
    joblib.dump(prophet_model, 'prophet_model.pkl')  # Prophet modelini kaydet  
except Exception as e:  
    print(f"Prophet model eğitimi sırasında hata: {e}")  

# ----- ARIMA Modeli -----  
try:  
    y_arima = grouped_data['conversion_count']  
    arima_model = ARIMA(y_arima, order=(5, 1, 0))  
    arima_result = arima_model.fit()  
    joblib.dump(arima_result, 'arima_model.pkl')  # ARIMA modelini kaydet  
except Exception as e:  
    print(f"ARIMA model eğitimi sırasında hata: {e}")  

# ----- LSTM Modeli -----  
try:  
    scaler = MinMaxScaler(feature_range=(0, 1))  
    scaled_data = scaler.fit_transform(grouped_data['conversion_count'].values.reshape(-1, 1))  

    # Eğitim ve test verileri  
    train_data = scaled_data[:-7]  
    x_train = []  
    y_train = []  
    for i in range(60, len(train_data)):  
        x_train.append(train_data[i-60:i, 0])  
        y_train.append(train_data[i, 0])  

    x_train, y_train = np.array(x_train), np.array(y_train)  
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))  

    # LSTM modelini oluştur  
    lstm_model = Sequential()  
    lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))  
    lstm_model.add(Dropout(0.2))  
    lstm_model.add(LSTM(units=50, return_sequences=False))  
    lstm_model.add(Dropout(0.2))  
    lstm_model.add(Dense(units=1))  

    # Modeli derle  
    lstm_model.compile(optimizer='adam', loss='mean_squared_error')  
    lstm_model.fit(x_train, y_train, epochs=100, batch_size=32)  
    lstm_model.save('lstm_model.h5')  # LSTM modelini kaydet  
except Exception as e:  
    print(f"LSTM model eğitimi sırasında hata: {e}")  

# ----- Değerlendirme Metrikleri -----  
def calculate_metrics(y_true, y_pred):  
    mape = mean_absolute_percentage_error(y_true, y_pred)  
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))  
    mae = mean_absolute_error(y_true, y_pred)  
    return {'MAPE': mape, 'RMSE': rmse, 'MAE': mae}  

# Sonuçları karşılaştırma  
y_true = grouped_data['conversion_count'].tail(7).values  
y_pred_prophet = prophet_model.predict(grouped_data.rename(columns={'date': 'ds'}))['yhat'].tail(7).values  
y_pred_arima = arima_result.forecast(steps=7).values  

# LSTM test verisi tahmini  
x_test = []  
for i in range(60, len(scaled_data)):  
    x_test.append(scaled_data[i-60:i, 0])  

x_test = np.array(x_test)  
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  
lstm_forecast = lstm_model.predict(x_test)  
lstm_forecast = scaler.inverse_transform(lstm_forecast)  
y_pred_lstm = lstm_forecast[-7:].flatten()  

# Metrikleri hesapla  
prophet_metrics = calculate_metrics(y_true, y_pred_prophet)  
arima_metrics = calculate_metrics(y_true, y_pred_arima)  
lstm_metrics = calculate_metrics(y_true, y_pred_lstm)  

# Metrikleri yazdır  
print("Prophet Metrikleri:", prophet_metrics)  
print("ARIMA Metrikleri:", arima_metrics)  
print("LSTM Metrikleri:", lstm_metrics)

c:\Users\Excalibur\Downloads\Etrn\Etrn\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
23:37:18 - cmdstanpy - INFO - Chain [1] start processing
23:37:18 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\Excalibur\Downloads\Etrn\Etrn\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\Excalibur\Downloads\Etrn\Etrn\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\Ex

Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - loss: 0.0068
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0046
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0062
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0068
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0094
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0046
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0050
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0082
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0043
Epoch 10/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0095
Epoch 11/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0044
Epoch 12/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0071
Epoch 13/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0072
Epoch 14/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0067
Epoch 15/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0046
Epoch 16/100
9/9 ━━

c:\Users\Excalibur\Downloads\Etrn\Etrn\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Excalibur\Downloads\Etrn\Etrn\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step
Prophet Metrikleri: {'MAPE': 0.1993884439556925, 'RMSE': np.float64(15.15174702791034), 'MAE': 12.773077067867234}
ARIMA Metrikleri: {'MAPE': 0.6632113572329299, 'RMSE': np.float64(32.65810190559549), 'MAE': 26.11984312656342}
LSTM Metrikleri: {'MAPE': 0.1986488401889801, 'RMSE': np.float64(17.56718930667962), 'MAE': 12.998663902282715}
